<a href="https://colab.research.google.com/github/Krdz14/FlashPointFireRescue/blob/main/APIPAPOI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install fastapi uvicorn nest_asyncio cloudflared
!apt-get install -y cloudflared

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
E: Unable to locate package cloudflared


In [ ]:
%%writefile main.py
from fastapi import FastAPI
import nest_asyncio

nest_asyncio.apply()

app = FastAPI()

last_state = {}
actions_queue = []

@app.get("/")
def home():
    return {"status": "running"}

# En tu main.py
state_history = []  # Lista para guardar historial
MAX_HISTORY = 100   # Límite de steps guardados

@app.post("/unity/send_state")
def receive_state(state: dict):
    global last_state, state_history
    last_state = state
    state_history.append(state)

    # Mantener solo los últimos 100 steps
    if len(state_history) > MAX_HISTORY:
        state_history.pop(0)

    return {"received": True, "total_steps": len(state_history)}

@app.get("/unity/get_state")
def get_state():
    return {"state": last_state}

@app.get("/unity/get_state/{step}")
def get_state_by_step(step: int):
    if 0 <= step < len(state_history):
        return {"state": state_history[step]}
    return {"error": "Step not found"}

@app.get("/unity/get_all_states")
def get_all_states():
    return {"states": state_history, "total": len(state_history)}

Writing main.py


In [ ]:
!ps -ef | grep uvicorn


root        1112       1  0 16:43 ?        00:00:00 /usr/bin/python3 /usr/local/bin/uvicorn main:app --host 0.0.0.0 --port 8003
root        1117     683  0 16:43 ?        00:00:00 /bin/bash -c ps -ef | grep uvicorn
root        1119    1117  0 16:43 ?        00:00:00 grep uvicorn


In [ ]:
!pkill -f uvicorn


In [ ]:
# Instalar cloudflared si no está
!curl -L https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 -o cloudflared
!chmod +x cloudflared
!mv cloudflared /usr/local/bin/

# Reiniciar servidor
!pkill -f uvicorn
!nohup uvicorn main:app --host 0.0.0.0 --port 8003 > output.log 2>&1 &
!sleep 3
!tail -n 20 output.log

# Abrir túnel
!cloudflared tunnel --url http://localhost:8003 --no-autoupdate

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 39.3M  100 39.3M    0     0  27.1M      0  0:00:01  0:00:01 --:--:-- 44.5M
INFO:     Started server process [1794]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8003 (Press CTRL+C to quit)
2025-12-04T16:49:09Z INF Thank you for trying Cloudflare Tunnel. Doing so, without a Cloudflare account, is a quick way to experiment and try it out. However, be aware that these account-less Tunnels have no uptime guarantee, are subject to the Cloudflare Online Services Terms of Use (https://www.cloudflare.com/website-terms/), and Cloudflare reserves the right to investigate your use of Tunnels for violati